## Importación de librerias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

## Carga de archivos

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/Bootcamp-IA-P4/flight-feel-analyzer/refs/heads/dev/data/airline_passenger_satisfaction.csv")

## Exploración Inicial

In [ ]:
df.head(10)

In [ ]:
print('La forma del dataset es', df.shape)
print('Rows duplicadas en train dataset :', df.duplicated().sum() )

In [ ]:
df.info()

In [ ]:
%%capture
!pip install ydata-profiling
from ydata_profiling import ProfileReport
from IPython.display import HTML # import the HTML object

In [ ]:
profile_change = ProfileReport(df,title="Airlines Satisfaction. Profiling Report", explorative=True,)
profile_change.to_file("output_change.html")
HTML(filename='output_change.html')

In [ ]:
df.describe()

## Limpieza

In [ ]:
df = df.drop(df.iloc[:,[0, 1]], axis = 1)

Aqui borramos las primeras 2 columnas (unnamed y id) porque no aportan valor

### Missing values

In [ ]:
correlation_arrival = df.corr(numeric_only=True)['Arrival Delay in Minutes'].sort_values(ascending=False)
print(correlation_arrival)

Vemos la relación que tiene la variable con missing data (Arrival Delay in Minutes) con otras

In [ ]:
plt.scatter(df['Arrival Delay in Minutes'], df['Departure Delay in Minutes'], alpha = 0.5)
pass

In [ ]:
from sklearn.linear_model import LinearRegression

# Separar los datos donde no hay NaN (para entrenar el modelo)
train_data = df.dropna(subset=['Arrival Delay in Minutes'])
X_train = train_data[['Departure Delay in Minutes']]
y_train = train_data['Arrival Delay in Minutes']

# Entrenar modelo lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir los valores faltantes
missing_mask = df['Arrival Delay in Minutes'].isnull()
X_missing = df.loc[missing_mask, ['Departure Delay in Minutes']]
df.loc[missing_mask, 'Arrival Delay in Minutes'] = model.predict(X_missing)


Imputamos los datos faltantes con una regresión lineal tomando como la variable predictora el Departure Delay in Minutes que es la que tiene una correlación de casi 0,97.

In [ ]:
df.isna().sum()

### Outliers

In [ ]:
def calculate_outliers_percentage(df):
    # Filtrar solo columnas numéricas
    df_numeric = df.select_dtypes(include='number')

    # Calcular los cuartiles y el IQR
    Q1 = df_numeric.quantile(0.25)
    Q3 = df_numeric.quantile(0.75)
    IQR = Q3 - Q1

    # Límites para detectar outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Contar los outliers por columna
    num_outliers = ((df_numeric < lower_bound) | (df_numeric > upper_bound)).sum()

    # Calcular el porcentaje de outliers
    pct_outliers = round(num_outliers / len(df) * 100, 4)

    return pct_outliers

In [ ]:
outliers = pd.DataFrame(calculate_outliers_percentage(df), columns=['% Outliers'])
outliers.index.name = 'Column Name'
outliers.reset_index(inplace=True)
outliers

Se pueden ver un gran número de valores atípicos en los retrasos y servicio check-in, y algunos en la distancia de vuelo. No obstante, en estos casos no podemos considerar que estos valores no sean válidos, ya que es factible que haya habido puntualmente algún retraso muy exagerado o una situación de colapso en check-in.

### Encoding de variables categóricas

Codificamos las variables categóricas que tienen unicamente dos valores como binarios.

In [ ]:
df['Gender'] = np.where(df['Gender'] == 'Male', 1, 0).astype('int16')
df['Customer Type'] = np.where(df['Customer Type'] == 'Loyal Customer', 1, 0).astype('int16')
df['Type of Travel'] = np.where(df['Type of Travel'] == 'Business Travel', 1, 0).astype('int16')

Encoding de la variable dependiente

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
col_encoded = le.fit_transform(df['satisfaction'])
df['satisfaction'] = col_encoded
df['satisfaction'].unique()

Label Encoder guarda una relación entre el número y la etiqueta original y permite revertir la codificación (inverse_transform) para interpretar resultados. Es buena práctca para la codificación de una variable dependiente categórica.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(categories=[['Eco', 'Eco Plus', 'Business']])
df['Class'] = encoder.fit_transform(df[['Class']]).astype('int8')

Usamos un Ordinal Encoding para preservar la jerarquía implícita, ya que Business > Eco Plus > Eco en términos de calidad/precio/servicio. Es una variable con orden, si usasemos One-Hot Encoding se perdería la jerarquía y los modelos no podrán captar que Business > Eco Plus > Eco, porque todas las clases se tratan como independientes

## Correlaciones después de la limpieza

In [ ]:
correlation=df.corr()
plt.figure(figsize=(14,8))
sns.heatmap(correlation,annot=True,fmt='.2f',annot_kws={'size': 10},linewidths=0.5,cmap='Blues')
plt.title("Data correlations after Preprocessing")

## Data Splitting

In [ ]:
X = df.drop(["satisfaction"], axis = 1)
y = df["satisfaction"]

Extraemos las features y la variable dependiente (la y, satifaction)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 1)

Dividimos el dataset en training y test, con proporción de 80% del training y 20% del test y stratificamos para mantener el balance de clases en ambos grupos.

In [ ]:
from sklearn.model_selection import StratifiedKFold

k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

Preparamos para implementar la validación cruzada:
- Partimos el dataset en 10 divisiones (9 para entrenar, 1 para validar, rotando).
- Mezclamos aleatoriamente los datos antes de dividirlos.
- Fijamos la semilla para que los resultados sean reproducibles.

## Desbalance

In [ ]:
y_train.value_counts(normalize=True)

Significa que el 56.7% de los ejemplos de entrenamiento pertenecen a la clase 0 ("neutral or dissatisfied") y el 43.3% a la clase 1 ("satisfied").
Está razonablemente balanceado, por lo que no es obligatorio aplicar técnicas de reamostrado (p.ej aumentando la clase minoritaria o disminuyendo la mayoritaria) o pesos(p.ej. penalizando más los errores en la clase minoritaria)

## Modelado

Como tenemos valores extremos pero reales (outliers reales en varias columnas) podría ser apropiado el uso de modelos que no se ven muy afectados por valores extremos, porque se basan en particiones (como árboles), como por ejemplo:
- Random Forest
- XGBoost / LightGBM / CatBoost
- Decision Tree
- Gradient Boosting

Los modelos como:
- Regresión logística
- SVM
- KNN

se ven muy afectados por los outliers.

Es cierto que, teniendo en cuenta el heatmap de correlación post-limpieza podríamos no tomar muy en cuenta las variables como los retrasos de salida/llegada ya que al parecer no tienen una correlación fuerte con la satisfacción (aunque podría parecer ligeramente sospechoso..), pero las columnas del servicio servicio check-in y la distancia de vuelo que también presentan outliers sí que muestran una correlación ligera-media con la satisfacción. En caso de trabajar con ellas, habría que hacer un escalado --> mejor con RobustScaler (por tener outliers, el RS usa el rango IQR)